In [1]:
import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from konlpy.tag import Okt
import pyLDAvis.gensim
import pyLDAvis
import matplotlib.pyplot as plt

In [8]:
# Okt 토크나이저를 초기화합니다.
okt = Okt()

# 간단한 한국어 불용어 리스트를 정의합니다.
korean_stopwords = ["는", "은", "이", "가", "을", "를", "에서", "의", "으로", "하고", "도", "에", "한", "으로", "고",'.']

# 예제 문장들
documents = [
    "축구 경기는 90분 동안 진행된다.",
    "야구에서는 홈런을 치면 관중들이 열광한다.",
    "테니스는 네트를 사이에 두고 상대와 경기를 한다.",
    "농구는 5명의 선수로 이루어진 팀 경기이다.",
    "마라톤은 장거리를 달리는 육상 경기이다.",
    "로맨틱 영화는 사랑 이야기를 중심으로 전개된다.",
    "액션 영화에서는 스릴 넘치는 추격 장면이 자주 등장한다.",
    "애니메이션 영화는 어린이들에게 인기가 많다.",
    "공포 영화는 긴장감을 조성하여 관객을 무섭게 한다.",
    "코미디 영화는 웃음을 주는 요소가 가득하다."
]

# 명사만 추출하여 불용어 제거
texts = [
    [word for word in okt.nouns(document) if word not in korean_stopwords]
    for document in documents
]

In [ ]:
texts

In [ ]:
len(set([j for i in texts for j in i]))

In [10]:
# Gensim의 Dictionary와 Corpus를 생성합니다. =>{단어:ID,,,,}
id2word = corpora.Dictionary(texts)
# print(len(id2word))
# id2word.token2id #내용확인

In [ ]:
id2word.token2id

In [17]:
# 필터링된 단어로 Corpus를 업데이트합니다.
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
corpus

In [19]:
# 최적의 토픽 수를 찾기 위한 평가
perplexity_values = []
coherence_values = []
topic_range = range(2, 7)

for num_topics in topic_range:
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=100)
    perplexity_values.append(lda_model.log_perplexity(corpus))
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_values.append(coherence_model_lda.get_coherence())

In [ ]:
perplexity_values

In [ ]:
plt.plot(perplexity_values, marker='.')

In [ ]:
coherence_values

In [ ]:
plt.plot(coherence_values, marker='.')

In [ ]:
# 결과 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(topic_range, perplexity_values, marker='o')
plt.title('Perplexity Scores')
plt.xlabel('Number of Topics')
plt.ylabel('Perplexity')

plt.subplot(1, 2, 2)
plt.plot(topic_range, coherence_values, marker='o')
plt.title('Coherence Scores')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence')

plt.show()

In [ ]:
# 최적 토픽 수로 최종 모델 학습
optimal_topics = topic_range[coherence_values.index(max(coherence_values))]
# optimal_topics=2
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=optimal_topics, random_state=100)

# pyLDAvis 시각화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

In [ ]:
# # filter_extremes 메서드를 사용하여 지나치게 자주 등장하거나 거의 등장하지 않는 단어들을 제거합니다.
# # 최소 2개 이상의 문서에 등장하고, 전체 문서의 50% 이하에서 등장하는 단어만 남깁니다.
# # 분류할 단어가 많다면no_below는 값을 올리고, no_above는 값을 줄인다!
# id2word.filter_extremes(no_below=2, no_above=0.5)
# print(len(id2word))
# id2word.token2id